In [3]:
import sys
import os
#import random forrest from sklearn 
from sklearn.ensemble import RandomForestClassifier


sys.path.append(os.path.abspath('../../src'))

from utils import *
from data_cleaning import *
from pipeline import *

from sklearn.metrics import f1_score

In [4]:
df_X, df_y = load_data_train()

In [5]:
df_X = convert_to_object(df_X)

In [6]:
df_X, df_y = drop_duplicates(df_X,df_y)

In [7]:
outliers_ids = get_outliers_ids(df_X)

tot number of outliers: 10526
- geo_level_1_id - number of outliers: 0
- geo_level_2_id - number of outliers: 0
- geo_level_3_id - number of outliers: 0
- count_floors_pre_eq - number of outliers: 2439
- age - number of outliers: 1259
- area_percentage - number of outliers: 3811
- height_percentage - number of outliers: 2407
- count_families - number of outliers: 2325


In [8]:
df_X, df_y = drop_row(outliers_ids.tolist(), df_X, df_y)

In [10]:
df_y = df_y.drop(['building_id'], axis=1)
df_X = df_X.drop(['building_id'], axis=1)
numerical_feature = df_X.select_dtypes(include=['number']).columns.tolist()
categorical_feature = df_X.select_dtypes(include=['object']).columns.tolist()

In [11]:
standard_scaler_cols = numerical_feature
robust_scaler_cols = []
baseN_enc_cols = categorical_feature
ordinal_enc_cols = []
one_hot_cols = []

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

In [12]:
model = RandomForestClassifier(n_estimators=100, random_state=0)

pre_proccessor = pipeline_preprocessor(standard_scaler_cols=standard_scaler_cols,one_hot_cols=ordinal_enc_cols)
pipeline = classifier_pipeline(pre_proccessor, model)

In [13]:
model_fit = model_training(X_train,y_train,pipeline)

/Users/davide/miniconda3/envs/DA_env/lib/python3.13/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [14]:
y_pred = model_fit.predict(X_test)
score = f1_score(y_test, y_pred, average='micro')
print(score)

0.6934177052825107


In [24]:
# test the model
df_test = load_data_test()
df_test = convert_to_object(df_test)
df_test_id = df_test.pop('building_id')

In [19]:
test_predictions = model_fit.predict(df_test)
df_submission = pd.DataFrame({'building_id': df_test_id, 'damage_grade': test_predictions})

In [20]:
df_submission

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2


In [28]:
df_submission.to_csv('../../results/20250129_16_24_submission.csv', index=False)